In [24]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [25]:
nfl = pd.read_csv('../../data/nfl_game_by_game_raw_stats.csv')

In [26]:
# Pull out only the rushing defense related stats

pass_def = nfl.copy()
pass_def = pass_def[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'total_pass_attempts_allowed', 'passing_yards_allowed',
        'completions_allowed', 'passing_epa_allowed', 'pass_tds_allowed', 'air_yards_allowed', 'yards_after_catch_allowed', 'air_epa_allowed', 'yac_epa_allowed', 'avg_cpoe_allowed',
        'qb_epa_allowed', 'qb_rush_yards_allowed', 'qb_rushing_epa_allowed', 'qb_rush_tds_allowed', 'sacks', 'qb_hits', 'fumbles_forced',
        'fumbles_recovered', 'interceptions']]
pass_def = pass_def.drop_duplicates()
pass_def

,season,week,team,opponent,score,opponent_score,total_pass_attempts_allowed,passing_yards_allowed,completions_allowed,passing_epa_allowed,pass_tds_allowed,air_yards_allowed,yards_after_catch_allowed,air_epa_allowed,yac_epa_allowed,avg_cpoe_allowed,qb_epa_allowed,qb_rush_yards_allowed,qb_rushing_epa_allowed,qb_rush_tds_allowed,sacks,qb_hits,fumbles_forced,fumbles_recovered,interceptions
0,2014,1,ARI,LAC,18,17,36,238.0,21.0,0.799435,1.0,148.0,90.0,8.328671,10.101167,-6.609174,-1.749010,10.0,-2.548446,0.0,0.0,4.0,0.0,0.0,1.0
1,2014,1,ATL,NO,37,34,42,333.0,29.0,9.202585,1.0,141.0,192.0,1.711281,22.435846,-0.291977,13.906330,2.0,-0.387024,0.0,0.0,0.0,1.0,1.0,1.0
2,2014,1,BAL,CIN,16,23,39,303.0,25.0,11.252418,1.0,129.0,172.0,-3.613447,26.073280,-2.044219,7.354762,5.0,-3.897657,0.0,0.0,1.0,0.0,0.0,0.0
3,2014,1,BUF,CHI,23,20,49,349.0,34.0,-6.755498,2.0,159.0,190.0,3.427141,16.035978,2.864331,0.058386,0.0,0.000000,0.0,2.0,4.0,1.0,1.0,2.0
4,2014,1,CAR,TB,20,14,35,183.0,22.0,-10.437736,2.0,138.0,45.0,6.082325,4.351872,0.519191,-3.645375,27.0,2.277812,0.0,3.0,9.0,2.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,2022,13,PIT,ATL,19,16,24,167.0,13.0,-4.254942,1.0,124.0,43.0,3.620184,6.045297,-3.001390,-2.200088,17.0,2.054854,0.0,1.0,3.0,0.0,0.0,1.0
5642,2022,13,SEA,LA,27,23,26,178.0,14.0,-5.930140,0.0,90.0,88.0,2.096860,10.056645,-4.741285,-4.127626,34.0,1.802514,0.0,4.0,9.0,1.0,0.0,2.0
5643,2022,13,SF,MIA,33,17,34,295.0,18.0,-14.312125,2.0,175.0,120.0,10.684227,14.150137,-9.637134,-14.312125,0.0,0.000000,0.0,3.0,5.0,1.0,1.0,3.0
5645,2022,13,TEN,PHI,10,35,41,391.0,30.0,19.095734,3.0,219.0,172.0,11.499256,17.068985,13.292768,23.936577,12.0,4.840843,1.0,2.0,3.0,1.0,0.0,0.0


In [27]:
pass_def.median()
# len(rush_def[rush_def.total_rush_yards_allowed > 105])
# len(rush_def)

<ipython-input-27-80f05b671c0b>:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pass_def.median()


season                         2018.000000
week                              9.000000
score                            23.000000
opponent_score                   23.000000
total_pass_attempts_allowed      34.000000
passing_yards_allowed           246.000000
completions_allowed              22.000000
passing_epa_allowed               1.446469
pass_tds_allowed                  1.000000
air_yards_allowed               131.000000
yards_after_catch_allowed       110.000000
air_epa_allowed                   3.890894
yac_epa_allowed                  13.955589
avg_cpoe_allowed                  0.461698
qb_epa_allowed                    2.838821
qb_rush_yards_allowed             9.000000
qb_rushing_epa_allowed            0.172446
qb_rush_tds_allowed               0.000000
sacks                             2.000000
qb_hits                           5.000000
fumbles_forced                    1.000000
fumbles_recovered                 0.000000
interceptions                     1.000000
dtype: floa

In [28]:
# Removing ties 

pass_def = pass_def[pass_def.score != pass_def.opponent_score]

# Add column to determine if qb won the game or not 

# qb['qb_win'] = qb.apply(lambda x: 1 if x.score > x.opponent_score else 0, axis=1)

# Add columns to transform rushing stats

pass_def['passing_epa_per_attempt_allowed'] = pass_def.passing_epa_allowed/pass_def.total_pass_attempts_allowed
pass_def['yards_per_attempt_allowed'] = pass_def.passing_yards_allowed/pass_def.total_pass_attempts_allowed
pass_def['touchdowns_per_attempt_allowed'] = pass_def.pass_tds_allowed/pass_def.total_pass_attempts_allowed
pass_def['completion_percentage_allowed'] = pass_def.completions_allowed/pass_def.total_pass_attempts_allowed
# pass_def['total_qb_plays_allowed'] = pass_def.total_pass_attempts_allowed + pass_def.total_qb_rush_attempts_allowed
# pass_def['total_epa_per_play_allowed'] = pass_def.qb_epa_allowed/pass_def.total_qb_plays_allowed

# Going to do something similar to rb value system, where instead of predicting qb win you predict whether or not 
# opposing team scores less than 23 points

pass_def['pass_def_win'] = pass_def.apply(lambda x: 1 if x.opponent_score < 23 else 0, axis=1)

pass_def

<ipython-input-28-cbabd838be53>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_def['passing_epa_per_attempt_allowed'] = pass_def.passing_epa_allowed/pass_def.total_pass_attempts_allowed
<ipython-input-28-cbabd838be53>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_def['yards_per_attempt_allowed'] = pass_def.passing_yards_allowed/pass_def.total_pass_attempts_allowed
<ipython-input-28-cbabd838be53>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,season,week,team,opponent,score,opponent_score,total_pass_attempts_allowed,passing_yards_allowed,completions_allowed,passing_epa_allowed,pass_tds_allowed,air_yards_allowed,yards_after_catch_allowed,air_epa_allowed,yac_epa_allowed,avg_cpoe_allowed,qb_epa_allowed,qb_rush_yards_allowed,qb_rushing_epa_allowed,qb_rush_tds_allowed,sacks,qb_hits,fumbles_forced,fumbles_recovered,interceptions,passing_epa_per_attempt_allowed,yards_per_attempt_allowed,touchdowns_per_attempt_allowed,completion_percentage_allowed,pass_def_win
0,2014,1,ARI,LAC,18,17,36,238.0,21.0,0.799435,1.0,148.0,90.0,8.328671,10.101167,-6.609174,-1.749010,10.0,-2.548446,0.0,0.0,4.0,0.0,0.0,1.0,0.022207,6.611111,0.027778,0.583333,1
1,2014,1,ATL,NO,37,34,42,333.0,29.0,9.202585,1.0,141.0,192.0,1.711281,22.435846,-0.291977,13.906330,2.0,-0.387024,0.0,0.0,0.0,1.0,1.0,1.0,0.219109,7.928571,0.023810,0.690476,0
2,2014,1,BAL,CIN,16,23,39,303.0,25.0,11.252418,1.0,129.0,172.0,-3.613447,26.073280,-2.044219,7.354762,5.0,-3.897657,0.0,0.0,1.0,0.0,0.0,0.0,0.288524,7.769231,0.025641,0.641026,0
3,2014,1,BUF,CHI,23,20,49,349.0,34.0,-6.755498,2.0,159.0,190.0,3.427141,16.035978,2.864331,0.058386,0.0,0.000000,0.0,2.0,4.0,1.0,1.0,2.0,-0.137867,7.122449,0.040816,0.693878,1
4,2014,1,CAR,TB,20,14,35,183.0,22.0,-10.437736,2.0,138.0,45.0,6.082325,4.351872,0.519191,-3.645375,27.0,2.277812,0.0,3.0,9.0,2.0,1.0,2.0,-0.298221,5.228571,0.057143,0.628571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,2022,13,PHI,TEN,35,10,26,157.0,16.0,-11.814701,1.0,96.0,61.0,-0.437060,6.091754,-4.796222,-6.407131,34.0,5.407570,0.0,6.0,7.0,0.0,0.0,0.0,-0.454412,6.038462,0.038462,0.615385,1
5641,2022,13,PIT,ATL,19,16,24,167.0,13.0,-4.254942,1.0,124.0,43.0,3.620184,6.045297,-3.001390,-2.200088,17.0,2.054854,0.0,1.0,3.0,0.0,0.0,1.0,-0.177289,6.958333,0.041667,0.541667,1
5642,2022,13,SEA,LA,27,23,26,178.0,14.0,-5.930140,0.0,90.0,88.0,2.096860,10.056645,-4.741285,-4.127626,34.0,1.802514,0.0,4.0,9.0,1.0,0.0,2.0,-0.228082,6.846154,0.000000,0.538462,0
5643,2022,13,SF,MIA,33,17,34,295.0,18.0,-14.312125,2.0,175.0,120.0,10.684227,14.150137,-9.637134,-14.312125,0.0,0.000000,0.0,3.0,5.0,1.0,1.0,3.0,-0.420945,8.676471,0.058824,0.529412,1


In [29]:
# Function to get relevant columns for modeling

# Note 1: Not using air yards or yac yards at the moment - since it's just predicting a win it won't be any different than 
# just the total passing yards idt. Might want a future adjustment to maybe raise/lower qb value based on air yards vs yac 
# yards pct

# Note 2: Going to use passing epa and qb rush epa seperately, rather than total qb epa

# Note 3: Using qb sacks and qb hits as predictive features. Not totally on the qb, but maybe theres some logic to 
# some qbs hold the ball too long and take too many sacks
# UPDATE: No longer using qb sacks and qb hits since they are pretty big factors in the qb value model, and since they are
# not entirely on the qb i think its not fair to add that, maybe worth adding in a separate linemen grading category

from pass_defense_value_helper_functions import get_features, get_label

In [30]:
# Test Sets of last 8 weeks of 2021 season and whatever we have from 2022 season

# test_2021 = qb[(qb.season == 2021) & (qb.week >= 9)]
# test_2021_x = get_features(test_2021)
# test_2021_y = get_label(test_2021)

# test_2022 = qb[qb.season == 2022]
# test_2022_x = get_features(test_2022)
# test_2022_y = get_label(test_2022)

test = pass_def[((pass_def.season == 2021) & (pass_def.week >= 9)) | (pass_def.season == 2022)]
test_x = get_features(test)
test_y = get_label(test)

train = pass_def[(pass_def.season < 2021) | ((pass_def.season == 2021) & (pass_def.week < 9))]
train_features = get_features(train)
train_labels = get_label(train)

pass_def.to_csv('pass_defense_value_data/pass_def.csv')
train.to_csv('pass_defense_value_data/train.csv')
test.to_csv('pass_defense_value_data/test.csv')

train_x, val_x, train_y, val_y = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

In [31]:
def train_model(model, train_x, train_y, val_x, val_y):
    clf = model.fit(train_x, train_y.to_numpy().flatten())
    
    train_preds = clf.predict(train_x)
    val_preds = clf.predict(val_x)
    
    train_acc = accuracy_score(train_y, train_preds)
    val_acc = accuracy_score(val_y, val_preds)
    
    print('Training Accuracy: {}'.format(train_acc))
    print('Validation Accuracy: {}'.format(val_acc))
    
    return clf

def test_model(clf, test_x, test_y):
    preds = clf.predict(test_x)
    probs = clf.predict_proba(test_x)[:,1]
    
    acc = accuracy_score(test_y, preds)
    
    print('Testing Accuracy: {}'.format(acc))
    
    return preds, probs

In [32]:
# Logistic Regression Model

model = LogisticRegression(max_iter=1000, random_state=71)
lr_model = train_model(model, train_x, train_y, val_x, val_y)

test_preds, test_probs = test_model(lr_model, test_x, test_y)

dump(lr_model, 'pass_defense_value_saved_models/logistic_regression.joblib')

Training Accuracy: 0.762669184765502
Validation Accuracy: 0.7710691823899372
Testing Accuracy: 0.751412429378531


['pass_defense_value_saved_models/logistic_regression.joblib']